In [17]:
import re
import math

import pandas as pd 
import numpy as np 
import seaborn as sns 

from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras import regularizers
from keras.activations import relu, sigmoid
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model

from scipy import stats
from matplotlib import pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

import warnings
pd.set_option('max_columns', None)
pd.set_option('display.float_format', str)
warnings.filterwarnings('ignore')
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [9,5]
plt.rcParams["axes.edgecolor"] = "black"

# Deep Learning notebook

#### Check if GPU exists

In [18]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print(tf.__version__)
print(f"Num GPUs Available: {len(tf.config.experimental.list_physical_devices('GPU'))}")
tf.test.is_gpu_available()
!python --version

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12829170627596824874
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 15729868357580525419
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7052715648
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5022602375352459211
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:06:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 10620902069311865642
physical_device_desc: "device: XLA_GPU device"
]
2.2.0
Num GPUs Available:  1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Python 3.6.13 :: Anaconda, Inc.


In [20]:
# Check if GPU is being used for training
# If not True then not
tf.config.experimental.get_memory_growth(physical_device[0])

In [21]:
# Set GPU to training 
tf.config.experimental.set_memory_growth(physical_device[0],True)

In [22]:
# Check again
tf.config.experimental.get_memory_growth(physical_device[0])

True

## Data

In [24]:
df = pd.read_csv('datasets/training/train_normlog.csv')
df.head()

,number,price,num_digits,unique_digits,num_zeros,max_group,max_group_second,shan_entrop,lv_dist,num_ones,num_twos,num_threes,num_fours,num_fives,num_sixes,num_sevens,num_eights,num_nines,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,0.012518888410771648,11.599112335481124,0.6666666666666666,0.25,0.0,0.25,0.25,0.43067655807339295,0.14123711340206135,0.4,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.031622451940878024,11.184435286790633,0.6666666666666666,0.25,0.0,0.5,0.0,0.3493984702822971,0.5324742268041227,0.0,0.75,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.2999229452911567,10.085850775128716,1.0,0.25,0.5,0.5,0.25,0.4181656600790517,0.7855670103092773,0.0,0.0,0.6000000000000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.07708473015841248,9.770013301136158,0.6666666666666666,0.5,0.0,0.25,0.0,0.6460148371100896,0.22989690721649406,0.0,0.0,0.0,0.0,0.25,0.25,0.6666666666666666,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.08820262386294267,9.770013301136158,0.6666666666666666,0.5,0.0,0.25,0.0,0.6460148371100896,0.28608247422680394,0.0,0.0,0.0,0.0,0.0,0.25,0.3333333333333333,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
